In [1]:
import math as math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

In [3]:
import route_dynamics.route_elevation.base_df as base
import route_dynamics.route_energy.longi_dynam_model as ldm
from route_dynamics.route_riders import route_riders as ride
from route_dynamics.route_visualizer import visualizer as vis

/Users/ericaeggleton/opt/miniconda3/envs/route_dynamics/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (46) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
shapefile_name = '../data/rt22_pts.shp'

In [5]:
data = pd.read_csv("../data/acceleration.csv", names=['time (s)', 'accel. (g)'])

In [6]:
data['time (s)'] = data['time (s)'].round(0)
data['accel. (g)'] = data['accel. (g)'].round(4)

In [7]:
#Convert to SI units
data['accel. (m/s^2)'] = data['accel. (g)']*9.81

In [8]:
#Calculate Velocity

data['vel. (m/s)'] = np.zeros(len(data.index))

for i in range(1, len(data)):
    data['vel. (m/s)'][i] = ((data['accel. (m/s^2)'][i] + data['accel. (m/s^2)'][i-1])/2*1) + data['vel. (m/s)'][i-1]
    
data['vel. (mph)'] = data['vel. (m/s)']*2.2

In [9]:
#Calculate Distance

data['dist. (m)'] = np.zeros(len(data.index))

for i in range(1, len(data)):
    data['dist. (m)'][i] = ((data['vel. (m/s)'][i] + data['vel. (m/s)'][i-1])/2*1) + data['dist. (m)'][i-1]

In [10]:
data['dist (ft)'] = data['dist. (m)']*3.28
data.head()

,time (s),accel. (g),accel. (m/s^2),vel. (m/s),vel. (mph),dist. (m),dist (ft)
0,0.0,0.0006,0.005886,0.000000,0.000000,0.000000,0.000000
1,1.0,0.1156,1.134036,0.569961,1.253914,0.284981,0.934736
2,2.0,0.1239,1.215459,1.744709,3.838359,1.442315,4.730794
3,3.0,0.1261,1.237041,2.970959,6.536109,3.800149,12.464488
4,4.0,0.1256,1.232136,4.205547,9.252203,7.388402,24.233957


In [11]:
#Create new dataframe where dist. is the indep var

data2 = pd.DataFrame({'dist. (ft)': np.arange(0, 1230, 6)})
data2['dist. (m)'] = data2['dist. (ft)']/3.28

data2['vel. (m/s)'] = np.zeros(len(data2.index))

data2['vel. (mph)'] = np.zeros(len(data2.index))

data2['accel. (m/s^2)'] = np.zeros(len(data2.index))

data2['time (s)'] = np.zeros(len(data2.index))

In [12]:
z = data2['dist. (ft)'].iloc[2]
params = data.iloc[(data['dist (ft)']-z).abs().argsort()[:1]]
row = params.index.values

#linear interpolation
vel = ((data['vel. (m/s)'][row].values - data['vel. (m/s)'][row-1].values)/(data['dist. (m)'][row].values - data['dist. (m)'][row-1].values) * (z/3.28 - data['dist. (m)'][row-1].values)) + data['vel. (m/s)'][row-1].values


accel = ((data['accel. (m/s^2)'][row].values - data['accel. (m/s^2)'][row-1].values)/(data['dist. (m)'][row].values - data['dist. (m)'][row-1].values) * (z/3.28 - data['dist. (m)'][row-1].values)) + data['accel. (m/s^2)'][row-1].values


In [13]:
for i in range(1, len(data2)):

    z = data2['dist. (ft)'].iloc[i]
    params = data.iloc[(data['dist (ft)']-z).abs().argsort()[:1]]
    row = params.index.values


    #Interpolate
    if params['dist (ft)'].values < z:
        vel = ((data['vel. (m/s)'][row+1].values - data['vel. (m/s)'][row].values)/(data['dist. (m)'][row +1].values - data['dist. (m)'][row].values) * (z/3.28 - data['dist. (m)'][row].values)) + data['vel. (m/s)'][row].values
        accel = ((data['accel. (m/s^2)'][row+1].values - data['accel. (m/s^2)'][row].values)/(data['dist. (m)'][row +1].values - data['dist. (m)'][row].values) * (z/3.28 - data['dist. (m)'][row].values)) + data['accel. (m/s^2)'][row].values
        time = ((data['time (s)'][row+1].values - data['time (s)'][row].values)/(data['dist. (m)'][row +1].values - data['dist. (m)'][row].values) * (z/3.28 - data['dist. (m)'][row].values)) + data['time (s)'][row].values
    else:
        vel = ((data['vel. (m/s)'][row].values - data['vel. (m/s)'][row-1].values)/(data['dist. (m)'][row].values - data['dist. (m)'][row-1].values) * (z/3.28 - data['dist. (m)'][row-1].values)) + data['vel. (m/s)'][row-1].values
        accel = ((data['accel. (m/s^2)'][row].values - data['accel. (m/s^2)'][row-1].values)/(data['dist. (m)'][row].values - data['dist. (m)'][row-1].values) * (z/3.28 - data['dist. (m)'][row-1].values)) + data['accel. (m/s^2)'][row-1].values
        time = ((data['time (s)'][row].values - data['time (s)'][row-1].values)/(data['dist. (m)'][row].values - data['dist. (m)'][row-1].values) * (z/3.28 - data['dist. (m)'][row-1].values)) + data['time (s)'][row-1].values
        
    
    data2['vel. (m/s)'].iloc[i] = vel
    data2['vel. (mph)'].iloc[i] = vel*2.2
    data2['accel. (m/s^2)'].iloc[i] = accel
    data2['time (s)'].iloc[i] = time

/Users/ericaeggleton/opt/miniconda3/envs/route_dynamics/lib/python3.6/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
data2.head()

,dist. (ft),dist. (m),vel. (m/s),vel. (mph),accel. (m/s^2),time (s)
0,0,0.000000,0.000000,0.000000,0.000000,0.000000
1,6,1.829268,1.945953,4.281097,1.219001,2.164114
2,12,3.658537,2.897310,6.374081,1.235745,2.939940
3,18,5.487805,3.551620,7.813564,1.234734,3.470328
4,24,7.317073,4.181005,9.198212,1.232234,3.980122


In [15]:
rt_df = base.wrapper(shapefile_name)

In [16]:
df_22, riders_22, mass_22 = ride.route_ridership('PM', 'I', 22) 
stop_coord, rider_coord = ride.stop_coord(22, riders_22)
coords22 = rider_coord['coordinates'].values
mass22 = rider_coord['Mean'].values

In [17]:
ex22 = ldm.RouteTrajectory(
    22, 
    shapefile_name, 
    bus_speed_model='const_accel_between_stops_and_speed_lim',
    stop_coords=coords22,
    mass_array=mass22 + mass_22,
    a_prof = "../data/acceleration.csv"
)

/Users/ericaeggleton/opt/miniconda3/envs/route_dynamics/lib/python3.6/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [18]:
rdf = ex22.route_df

In [19]:
rdf.tail()

,Z,geometry,length,grade,distance_from_last_point,elevation,cum_distance,is_bus_stop,acceleration,velocity,delta_time,mass,grav_force,roll_fric,aero_drag,inertia,power_output
9714,324.334638,POINT (-122.38813 47.56285),58284,0.038188,1.8288,324.334638,17764.9632,False,0.638199,13.414576,0.139686,12947.0,-4846.736988,-1269.175599,-23.968962,8262.765386,193205.396321
9715,324.566006,POINT (-122.38813 47.56288),58290,0.038561,1.8288,324.566006,17766.7920,False,0.632491,13.497637,0.133877,12947.0,-4894.040649,-1269.157447,-24.117376,8188.860615,194044.412001
9716,324.799624,POINT (-122.38813 47.56290),58296,0.038936,1.8288,324.799624,17768.6208,False,0.626741,13.579705,0.133212,12947.0,-4941.549253,-1269.139038,-24.264013,8114.416008,194860.186773
9717,325.035500,POINT (-122.38813 47.56292),58302,0.039313,1.8288,325.035500,17770.4496,False,0.620991,13.661772,0.133212,12947.0,-4989.262754,-1269.120370,-24.410650,8039.971401,195674.357167
9718,325.273646,POINT (-122.38813 47.56295),58308,0.039691,1.8288,325.273646,17772.2784,False,0.615241,13.743840,0.133212,12927.0,-5029.399870,-1267.140985,-24.557287,7953.221972,196183.969156


In [20]:
delta_t = rdf.delta_time.values[1:]
delta_t

array([2.16411381, 0.7758259 , 0.5303884 , ..., 0.13321171, 0.13321171,
       0.13321171])

In [21]:
power = rdf.power_output.values[1:]
power

array([ 27357.10175277,  41396.81367747,  50743.8062015 , ...,
       194860.18677293, 195674.35716691, 196183.96915603])

In [ ]:
test = power*delta_t

In [ ]:
np.sum(power*delta_t)

In [22]:
energy = np.sum(power * delta_t)

In [23]:
energy

122645248.69623792

In [24]:
ex22.energy_from_route() 

122645248.69623792

In [ ]:
v_lim = 15 #m/s or ~30 mph
a_neg = -0.4

# Define cutoff distance for acceleration and deceleration
a_avg = data2['accel. (m/s^2)'].mean()
a_plt = data2['accel. (m/s^2)'].iloc[len(data2)-1]
x_a = v_lim**2. / (2*a_avg)

# Define cutoff distance for acceleration and deceleration
x_d = v_lim**2. / (2*a_neg)

# Distance of accel profile

x_p = len(data2)*0.3048


In [ ]:
x_ns = np.zeros(len(route_df.index)) #next stops
x_ls = np.zeros(len(route_df.index)) # prev. stops

for i in range(len(x_ns)):
    # set values to Nan if bus stop
    if route_df.at[i, 'is_bus_stop']:
        x_ns[i] = 0.
        x_ls[i] = 0.
        # move to next point
        continue
    else:
        # Calculate 'x_ns';
        # Iterate through remaining indicies to count distance to
        # next stop.
        for j in range(i+1, len(x_ns)):
            # add distance to next point to 'x_ns'
            x_ns[i] += route_df.at[j, 'distance_from_last_point']
            if route_df.at[j, 'is_bus_stop']:
                break # done calulating 'x_ns' at this point
            # elif not bus stop: move to next point, add distance

        # Calculate 'x_ls';
        # Iterate through previous indicies to cout distance to
        # last stop.
        for j in range(i, 0, -1):
            # Inclusive start to range because distances are
            # backward difference. Dont need to include 'j=0'
            # because the first point has no backward difference.
            if route_df.at[j, 'is_bus_stop']:
                break # done calulating x_ls at this point
            x_ls[i] += route_df.at[j, 'distance_from_last_point']


In [ ]:
v = np.zeros(len(route_df.index)) #array for vel.
a = np.zeros(len(route_df.index)) #array for accel.

count = 0

for i in range(len(x_ns)):

    if count > i:
        continue

    else:

        #Case 1

        if (
            x_ns[i]<=abs(x_d)
            and
            not route_df.at[i, 'is_bus_stop']
            ):
            
            if x_ns[i] < x_ls[i]:

                a[i] = a_neg
                v[i] = np.sqrt(-2*x_ns[i]*a_neg)
                count += 1
                
            else:
                for j in range(len(data2)):

                    if count < len(x_ns):

                        if v[count-1] < v_lim:
                            a[count] = data2['accel. (m/s^2)'].iloc[j]
                            v[count] = data2['vel. (m/s)'].iloc[j]
                            count+=1

                        elif v[count-1]>v_lim:

                            while x_ns[count]>abs(x_d):

                                a[count] = 0
                                v[count] = v_lim

                                count+=1

                            else:
                                continue

                        else:
                            continue

                    else:
                        break


        #Case 2

        elif (
            x_ns[i] > abs(x_d)
            and
            x_ls[i] >= x_p
            and
            not route_df.at[i, 'is_bus_stop']
            ):

            if v[i-1] < v_lim:

                a[i] = a_plt
                v[i] = np.sqrt(2*x_ns[i]*a[i])
                count += 1

            elif v[i-1]>v_lim:
                        
                while x_ns[count]>abs(x_d):

                    a[count] = 0
                    v[count] = v_lim

                    count+=1

                else:
                    continue 

            else:
                continue

                    

        #Case 3

        elif (
            x_ls[i] < x_p
            and
            x_ns[i] > abs(x_d)
            and
            not route_df.at[i, 'is_bus_stop']
            ):

            for j in range(len(data2)):

                if count < len(x_ns):

                    if v[count-1] < v_lim:
                        a[count] = data2['accel. (m/s^2)'].iloc[j]
                        v[count] = data2['vel. (m/s)'].iloc[j]
                        count+=1

                    elif v[count-1]>v_lim:
                        
                        while x_ns[count]>abs(x_d):
                
                            a[count] = 0
                            v[count] = v_lim

                            count+=1
                            
                        else:
                            continue
                            
                    else:
                        continue

                else:
                    break


        elif route_df.at[i, 'is_bus_stop']:

            count += 1

In [ ]:
ex22.route_df

In [ ]:
d = {'dist': ex22.route_df.cum_distance, 'last': x_ls, 'next': x_ns, 'stop': ex22.route_df.is_bus_stop, 'accel': a, 'vel':v}

In [ ]:
test = pd.DataFrame(d)

In [ ]:
print (x_d, x_p)

In [ ]:
test.iloc[330:370]

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (8,6), dpi=100)

ax1.plot(ex22.route_df.cum_distance, v, label = 'vel')
ax2.plot(ex22.route_df.cum_distance, a, label = 'accel')
#ax.set_xlabel('time (s)') 
#ax.set_ylabel('Power (kW)')
#ax.legend()